# Baseline Experiments

In [4]:
import time
import datetime

import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
from torchvision import models

from tqdm.notebook import tqdm

## Load Data

In [ ]:
# TODO: create datasets, dataloaders, etc

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

## Training

In [ ]:
WEIGHTS = "imagenet"
N_CLASSES = 2

In [ ]:
def format_time(elapsed):
  '''
    Takes a time in seconds and returns a string hh:mm:ss
  '''
  
  elapsed_rounded = int(round((elapsed))) # Round to the nearest second
  return str(datetime.timedelta(seconds=elapsed_rounded)) # Format as hh:mm:ss

In [ ]:
def train_model(model, dataloaders, dataset_sizes, criterion, optimizer, scheduler, n_epochs=25):
  '''
    General function to train a model
  '''

  # Measure the total training time for the whole run
  total_t0 = time.time()

  best_model = model.state_dict()
  best_acc = 0.0

  for epoch_i in range(n_epochs):
    print()
    print('========== Epoch {:} / {:} =========='.format(epoch_i + 1, n_epochs))
    
    # Measure the training time for epoch
    t0 = time.time()

    best_model = model.state_dict()
    best_acc = 0.0

    # Each epoch has a training and validation phase
    for phase in ["train", "val"]:
      if phase == "train":
        print("Training...")
        model.train() # Set model to training mode
      else:
        print("Running Validation...")
        model.eval() # Set model to evaluate mode

      run_loss = 0.0
      run_corrects = 0

      # Iterate over data
      for inputs, labels in tqdm(dataloaders[phase]):
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward
        # Track history if only in train
        with torch.set_grad_enabled(phase == "train"):
          outputs = model(inputs)
          _, preds = torch.max(outputs, 1)
          loss = criterion(outputs, labels)

          # If in training phase, backward + optimize
          if phase == "train":
            loss.backward()
            optimizer.step()

        # Statistics 
        run_loss += loss.item() * inputs.size(0)
        run_corrects += torch.sum(preds == labels.data)

      if phase == "train":
        scheduler.step()

      epoch_loss = run_loss / dataset_sizes[phase]
      epoch_acc = run_corrects.double() / dataset_sizes[phase]

      print("{:} Loss: {:.4f}").format("Training" if phase == "train" else "Validation", epoch_loss)
      print("{:} Acc: {:.4f}").format("Training" if phase == "train" else "Validation", epoch_acc)

      if phase == "val":
        if epoch_acc > best_acc:
          best_acc = epoch_acc
          best_model = model.state_dict()
        
        print("Epoch took {:}".format(format_time(time.time() - t0)))

      print()

  print("Training complete!")
  print("Total training took {:}".format(format_time(time.time() - total_t0)))
  print("Best Acc: {:.4f}".format(best_acc))

  # Load best model
  model.load_state_dict(best_model)
  
  return model

In [ ]:
criterion = nn.CrossEntropyLoss()

def get_optimizer(model):
  # Parameters of frozen layers will not be optimized
  return optim.SGD(
      params=list(filter(lambda p: p.requires_grad, model.parameters())), 
      lr=0.001, 
      momentum=0.9
    )

def get_scheduler(optimizer):
  # Decay LR by a factor of 0.1 every 7 epochs
  return optim.lr_scheduler.StepLR(optimizer=optimizer, step_size=7, gamma=0.1)

### ResNet50

In [ ]:
resnet = models.resnet50(pretrained=WEIGHTS)

for params in resnet.parameters(): 
  params.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
n_features = resnet.fc.in_features
resnet.fc = nn.Linear(n_features, N_CLASSES)

optimizer = get_optimizer(resnet)
scheduler = get_scheduler(optimizer)

In [ ]:
# TODO: add missing parameters
# resnet = train_model(resnet, criterion, optimizer, scheduler)

### DenseNet121

In [6]:
densenet = models.densenet121(pretrained=WEIGHTS)

for params in densenet.parameters(): 
  params.requires_grad = False

n_features = densenet.classifier.in_features
densenet.classifier = nn.Linear(n_features, N_CLASSES)

optimizer = get_optimizer(densenet)
scheduler = get_scheduler(optimizer)

In [ ]:
# TODO: add missing parameters
# densenet = train_model(densenet, criterion, optimizer, scheduler)

### VGG16

In [ ]:
vgg = models.vgg16(pretrained=WEIGHTS)

for params in resnet.parameters(): 
  params.requires_grad = False

n_features = vgg.classifier[6].in_features
features = list(vgg.classifier.children())[:-1]
features.extend([nn.Linear(n_features, N_CLASSES)])
vgg.classifier = nn.Sequential(*features)

optimizer = get_optimizer(vgg)
scheduler = get_scheduler(optimizer)

In [ ]:
# TODO: add missing parameters
# vgg = train_model(vgg, criterion, optimizer, scheduler)